# 第七节 LangChain 表达式语言 (LCEL)_2

https://langchain114.com/docs/expression_language

https://blog.csdn.net/Attitude93/article/details/136531425

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

from langchain.chains import LLMChain

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser #仅仅是让输出对象成为字符串

from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain_community.embeddings import QianfanEmbeddingsEndpoint

from operator import itemgetter

In [2]:
model_ty = Tongyi(temperature=0.1)
model_qf = QianfanLLMEndpoint(temperature=0.1)
chat_ty = ChatTongyi()

## 自定义函数

RunnableLambda

https://langchain114.com/docs/expression_language/how_to/functions

In [6]:
from langchain_core.runnables import RunnableLambda


def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
chain1 = prompt | model_ty
chain =(
{
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
    | RunnableLambda(multiple_length_function),
}
| prompt
| model_ty
)

In [8]:
chain1.invoke({"a":3,"b":7})

'10'

In [9]:
chain.invoke({"foo":"bar","bar":"gah"})

'12'

## 根据输入动态路由逻辑

有两种执行路由的方式：

* 使用 RunnableBranch。
* 编写自定义工厂函数，该函数接受前一步的输入并返回一个 runnable。重要的是，这应该返回一个 runnable 而不是实际执行。

https://langchain114.com/docs/expression_language/how_to/routing

* 基于语义相似性的路由

https://langchain114.com/docs/expression_language/cookbook/embedding_router


In [13]:
from langchain.prompts import PromptTemplate
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

physics_template = """你是一个非常聪明的物理学教授。
你擅长以简洁易懂的方式回答物理问题。
当你不知道答案时，你会承认不知道。

这是一个问题：
{query}"""

math_template = """你是一个非常优秀的数学家。
你擅长回答数学问题。
你之所以优秀是因为你能够将难题分解为组成部分，
回答组成部分，然后把它们组合起来回答更广泛的问题。

这是一个问题：
{query}"""

embeddings = QianfanEmbeddingsEndpoint()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("使用数学" if most_similar == math_template else "使用物理")
    return PromptTemplate.from_template(most_similar)

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatTongyi()
    | StrOutputParser()
)

[INFO] [04-02 13:50:42] openapi_requestor.py:316 [t:3748]: requesting llm api endpoint: /embeddings/embedding-v1


In [12]:
len(prompt_embeddings)

2

In [14]:
print(chain.invoke("什么是黑洞"))

[INFO] [04-02 13:50:44] openapi_requestor.py:316 [t:3748]: requesting llm api endpoint: /embeddings/embedding-v1


使用物理
黑洞是宇宙中一种极其奇特的天体，它的存在基于爱因斯坦的广义相对论。简单来说，黑洞是由极度密集的物质，通常是恒星在死亡后塌缩形成的，其引力强大到连光也无法逃脱，因此得名“黑洞”（英文为Black Hole）。

在一个黑洞的事件视界内，任何东西一旦进入，就再也无法离开，包括光线。这个边界被称为“奇点”，其中物质的密度和引力场无限大。我们不能直接观察黑洞，因为它们不发射或反射光线，但我们可以通过测量周围物体的运动和辐射来推断它们的存在。

黑洞的研究是现代天体物理学的重要领域，它们帮助科学家们理解宇宙的极端条件和引力的强大力量。虽然我们不能亲眼看到黑洞，但通过理论计算和间接证据，科学家们正在逐步揭示它们的秘密。


In [15]:
print(chain.invoke("什么是路径积分"))

[INFO] [04-02 13:50:54] openapi_requestor.py:316 [t:3748]: requesting llm api endpoint: /embeddings/embedding-v1


使用数学
路径积分是量子力学和数学物理学中的一个重要概念，由理论物理学家理查德·费曼在20世纪40年代提出。它是一种计算量子系统中粒子行为的方法，尤其是在没有经典解析解的情况下。在量子力学中，路径积分可以理解为所有可能路径（从一个位置到另一个位置）的贡献之和，每个路径都有一个对应的复数概率幅度。

简单来说，路径积分可以看作是对路径空间进行的一种积分，这里的“路径”指的是粒子从一个点移动到另一个点的所有可能轨迹。在经典力学中，我们通常只考虑一条确定的路径，但在量子力学中，由于波粒二象性，粒子的行为表现为概率分布，所以我们需要考虑所有可能的路径并加权求和。

路径积分公式通常用于薛定谔方程的量子化形式，即所谓的费曼路径积分，它是量子场论的基础之一，对于量子电动力学、量子色动力学等现代粒子物理学理论具有关键作用。


### classwork 1

* 改写上面基于相似性路由代码，加入以为化学专家，是的程序能基于问题在三个专家中选择最符合的专家回答

## 基于LECL实现通过python程序来完成问题回答

* classwork2 完成此节代码

In [16]:
from langchain.utilities import PythonREPL

In [17]:
print(3*3)

9


In [18]:
PythonREPL().run('print(3*3)')

Python REPL can execute arbitrary code. Use with caution.


'9\n'

In [ ]:
# 提出问题——语言模型写代码回答问题——从回答中提取代码——运行代码=结果

In [34]:
template_code="请通过写python代码来回答对应问题，此问题是：{question} \n 注意仅需要通过python代码回答，不需要文字 \n 代码形式如下: ```python\n...\n```"
t_code=PromptTemplate.from_template(template_code)

def get_code(x):
    return x.split("```python")[1].split("```")[0]

chain_code=(t_code|ChatTongyi()|StrOutputParser()|RunnableLambda(get_code)|PythonREPL().run)

In [46]:
#chain_code.invoke({"question":"请帮我获取网页'https://abs.hznu.edu.cn/c/2024-03-29/2936058.shtml'中的内容"})

In [ ]:
print(chain_pro.invoke({"question":"2乘以8等于几"}))

In [57]:
import os
current_path = os.getcwd()

print(current_path)

C:\Users\Administrator


## 基于LCEL实现自动查询 SQL 数据库

* sqlite数据库下载

链接：https://pan.baidu.com/s/1_nvpBZcrZ_3dgYD1HKX3rg?pwd=kp0h 
提取码：kp0h

RunnablePassthrough.assign(schema=get_schema,response=lambda x: run_query(x["query"]))

* classwork3 完成此节代码

In [ ]:
# 基于对应数据提出问题——语言模型基于问题生成sql——提取sql——执行sql——综合所有信息生成模板——语言模型基于所有信息进行回答

In [40]:
from langchain_community.utilities import SQLDatabase

In [41]:
db = SQLDatabase.from_uri("sqlite:///ali_langchain.db")

def get_schema(_):
    return db.get_table_info()
    
def run_query(query):
    return db.run(query)

In [44]:
run_query("SELECT COUNT(*) FROM alidata")

'[(182880,)]'

In [45]:
#print(db.get_table_info())

In [53]:
template_sql="请通过写sql代码来回答对应问题，并且需要基于如下数据库信息：{info} \n 需要回答问题是：{question} \n 注意仅需要通过sql代码回答，不需要文字 \n 代码形式如下: ```sql\n...\n```"
t_sql=PromptTemplate.from_template(template_sql)

def get_sql(x):
    return x.split("```sql")[1].split("```")[0]

chain_sql=({"info":get_schema,"question":RunnablePassthrough()}|t_sql|ChatTongyi()|StrOutputParser()|RunnableLambda(get_sql))

template_sql0="请通过综合如下的数据库信息，问题，sql代码，sql代码的执行结果给出问题的自然语言回答：\n 数据库信息{info} \n 需要回答问题是：{question} \n sql代码：{query} \n sql代码执行结果: {res}"
t_sql0=PromptTemplate.from_template(template_sql0)

chain_sql0=({"info":get_schema,"question":RunnablePassthrough(),"query":chain_sql}|RunnablePassthrough.assign(res=lambda x: run_query(x["query"]))|t_sql0|ChatTongyi()|StrOutputParser())

In [56]:
chain_sql0.invoke("记录数最多的五个brand是")

'在alidata表中，记录数最多的五个品牌（按数量降序）分别是：\n\n1. 品牌7868，出现了3356次\n2. 品牌11196，出现了2147次\n3. 品牌11080，出现了1688次\n4. 品牌12220，出现了1682次\n5. 品牌21110，出现了1597次'

In [52]:
run_query("\nSELECT COUNT(*) FROM alidata;\n")

'[(182880,)]'

In [13]:
chain_sql2.invoke("表里有多少brand？")

AIMessage(content='在alidata表中，有9531个不同的品牌。')

In [127]:
run_query('\nSELECT COUNT(*) FROM alidata;\n')

'[(182880,)]'

参考链接：

https://langchain114.com/docs/expression_language/cookbook/sql_db

https://langchain114.com/docs/expression_language/cookbook/code_writing